- the purpose of this script is to show how some properties of the data behave locally

In [2]:
@time include("../startup_script.jl")

elapsed time: 0.494417748 seconds (52416884 bytes allocated, 8.67% gc time)
elapsed time: 52.062909832 seconds (5419744236 bytes allocated, 65.06% gc time)
elapsed time: 0.742616777 seconds (112671656 bytes allocated, 48.67% gc time)
elapsed time: 66.922377146 seconds (5419744236 bytes allocated, 72.34% gc time)
elapsed time: 140.922123209 seconds (14792109664 bytes allocated, 60.68% gc time)


,Date,ID,Bid,Ask,Volume,Open_Interest
1,2006-07-03,c_20061215_1800,NA,NA,1,104
2,2006-07-03,p_20061215_1800,NA,NA,0,5515
3,2006-07-03,c_20061215_2000,NA,NA,0,2152
4,2006-07-03,p_20061215_2000,NA,NA,0,20941
5,2006-07-03,c_20061215_2200,NA,NA,0,2
6,2006-07-03,p_20061215_2200,NA,NA,0,4626


- load data

In [3]:
putLiquData = readtable("../data/chart_data/putLiquidityData.csv")

head(putLiquData)

,Date,ID,Bid,Ask,Volume,Open_Interest,Expiry,Strike,IsCall,DAX,VolRanks,DeltaOI,OiRanks
1,2006-07-03,p_20060721_4200,NA,NA,0,1337,2006-07-21,4200,false,5712.69,0.0,1337.0,0.5011194343079209
2,2006-07-03,p_20060721_4300,NA,NA,0,3260,2006-07-21,4300,false,5712.69,0.0,3260.0,0.6202273434298045
3,2006-07-03,p_20060721_4400,NA,NA,0,5565,2006-07-21,4400,false,5712.69,0.0,5565.0,0.6946808353044002
4,2006-07-03,p_20060721_4500,NA,NA,0,4252,2006-07-21,4500,false,5712.69,0.0,4252.0,0.654867588747389
5,2006-07-03,p_20060721_4600,NA,NA,0,5335,2006-07-21,4600,false,5712.69,0.0,5335.0,0.6879854626616563
6,2006-07-03,p_20060721_4700,NA,NA,150,3380,2006-07-21,4700,false,5712.69,0.5902119355387291,3380.0,0.6248482813449013


- calculate moneyness

In [4]:
nObs = size(putLiquData, 1)

mnyness = zeros(nObs)
for ii=1:nObs
    mnyness[ii] = putLiquData[ii, :Strike] - putLiquData[ii, :DAX]
end

putLiquData[:mnyness] = mnyness

head(putLiquData)

,Date,ID,Bid,Ask,Volume,Open_Interest,Expiry,Strike,IsCall,DAX,VolRanks,DeltaOI,OiRanks,mnyness
1,2006-07-03,p_20060721_4200,NA,NA,0,1337,2006-07-21,4200,false,5712.69,0.0,1337.0,0.5011194343079209,-1512.6899999999996
2,2006-07-03,p_20060721_4300,NA,NA,0,3260,2006-07-21,4300,false,5712.69,0.0,3260.0,0.6202273434298045,-1412.6899999999996
3,2006-07-03,p_20060721_4400,NA,NA,0,5565,2006-07-21,4400,false,5712.69,0.0,5565.0,0.6946808353044002,-1312.6899999999996
4,2006-07-03,p_20060721_4500,NA,NA,0,4252,2006-07-21,4500,false,5712.69,0.0,4252.0,0.654867588747389,-1212.6899999999996
5,2006-07-03,p_20060721_4600,NA,NA,0,5335,2006-07-21,4600,false,5712.69,0.0,5335.0,0.6879854626616563,-1112.6899999999996
6,2006-07-03,p_20060721_4700,NA,NA,150,3380,2006-07-21,4700,false,5712.69,0.5902119355387291,3380.0,0.6248482813449013,-1012.6899999999996


- attach time to maturity

In [5]:
function convertColToDates!(df::DataFrame, col::Symbol)
    dats = Date(convert(Array, df[:, col]))
    df[col] = dats
end

@time convertColToDates!(putLiquData, :Date)
@time convertColToDates!(putLiquData, :Expiry)

elapsed time: 36.223402325 seconds (2776011524 bytes allocated, 73.04% gc time)
elapsed time: 34.738824491 seconds (2775935732 bytes allocated, 71.93% gc time)


1037251-element Array{Date,1}:
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 2006-07-21
 ⋮         
 2014-12-19
 2014-12-19
 2014-12-19
 2014-12-19
 2014-12-19
 2014-12-19
 2014-12-19
 2014-12-19
 2014-12-19
 2014-12-19
 2014-12-19
 2014-12-19

In [6]:
fullData = join(putLiquData, cohortParams, on = [:Date, :Expiry])

,Date,ID,Bid,Ask,Volume,Open_Interest,Expiry,Strike,IsCall,DAX,VolRanks,DeltaOI,OiRanks,mnyness,EONIA_matched,Time_to_Maturity
1,2006-07-03,p_20060721_4200,NA,NA,0,1337,2006-07-21,4200,false,5712.69,0.0,1337.0,0.5011194343079209,-1512.6899999999996,0.0283102283088403,0.0549019607843137
2,2006-07-03,p_20060721_4300,NA,NA,0,3260,2006-07-21,4300,false,5712.69,0.0,3260.0,0.6202273434298045,-1412.6899999999996,0.0283102283088403,0.0549019607843137
3,2006-07-03,p_20060721_4400,NA,NA,0,5565,2006-07-21,4400,false,5712.69,0.0,5565.0,0.6946808353044002,-1312.6899999999996,0.0283102283088403,0.0549019607843137
4,2006-07-03,p_20060721_4500,NA,NA,0,4252,2006-07-21,4500,false,5712.69,0.0,4252.0,0.654867588747389,-1212.6899999999996,0.0283102283088403,0.0549019607843137
5,2006-07-03,p_20060721_4600,NA,NA,0,5335,2006-07-21,4600,false,5712.69,0.0,5335.0,0.6879854626616563,-1112.6899999999996,0.0283102283088403,0.0549019607843137
6,2006-07-03,p_20060721_4700,NA,NA,150,3380,2006-07-21,4700,false,5712.69,0.5902119355387291,3380.0,0.6248482813449013,-1012.6899999999996,0.0283102283088403,0.0549019607843137
7,2006-07-03,p_20060721_4800,NA,NA,64,18200,2006-07-21,4800,false,5712.69,0.4646165140333419,18200.0,0.8744134944423321,-912.6899999999996,0.0283102283088403,0.0549019607843137
8,2006-07-03,p_20060721_4850,NA,NA,104,4130,2006-07-21,4850,false,5712.69,0.5447031269205448,4130.0,0.6515561963923304,-862.6899999999996,0.0283102283088403,0.0549019607843137
9,2006-07-03,p_20060721_4900,NA,NA,1282,20927,2006-07-21,4900,false,5712.69,0.8384932321053633,20927.0,0.8931352555144608,-812.6899999999996,0.0283102283088403,0.0549019607843137
10,2006-07-03,p_20060721_4950,NA,NA,0,4855,2006-07-21,4950,false,5712.69,0.0,4855.0,0.6742569612814979,-762.6899999999996,0.0283102283088403,0.0549019607843137


- define function to evaluate local properties

In [7]:
function localAppl(df::DataFrame, xCol::Symbol, yCol::Symbol,
                   xE::Array{Float64, 1}, yE::Array{Float64, 1},
                   aggrFunc::Function)

    ## calculate interval midpoints
    xMids = midpoints(xE)
    yMids = midpoints(yE)

    ## get grid dimensions
    n = length(xE) - 1
    m = length(yE) - 1

    ## preallocation
    xInds = zeros(Int64, nObs)
    yInds = zeros(Int64, nObs)
    
    # find respective grid interval
    for ii = 1:nObs
        x = searchsortedfirst(xE, df[ii, xCol]) - 1
        y = searchsortedfirst(yE, df[ii, yCol]) - 1

        xInds[ii] = x
        yInds[ii] = y
    end
    extDf = [df DataFrame(xInds = xInds, yInds = yInds)]

    # group by grid interval, apply function
    res = by(extDf, [:xInds, :yInds], aggrFunc)
    
    # translate interval indices to values
    nIntervals = size(res, 1)
    
    xVals = zeros(nIntervals)
    yVals = zeros(nIntervals)
    for ii=1:nIntervals
        xVals[ii] = xMids[res[ii, :xInds]]
        yVals[ii] = yMids[res[ii, :yInds]]
    end

    # rename results
    finalDf = DataFrame(x = xVals, y = yVals, vals = res[:x1])
    names!(finalDf, [xCol, yCol, :value])

    return finalDf

end


localAppl (generic function with 1 method)

- where are observations?

In [8]:
xEdges = [0:0.5:5]
yEdges = Float64[-10000:500:13000]

xCol = :Time_to_Maturity
yCol = :mnyness

function countObs(df::SubDataFrame)
    return size(df, 1)
end

countObs (generic function with 1 method)

In [9]:
res = localAppl(fullData, xCol, yCol, xEdges, yEdges, countObs)

,Time_to_Maturity,mnyness,value
1,0.25,-8750.0,2
2,0.25,-8250.0,52
3,0.25,-7750.0,77
4,0.25,-7250.0,261
5,0.25,-6750.0,471
6,0.25,-6250.0,1230
7,0.25,-5750.0,1980
8,0.25,-5250.0,2932
9,0.25,-4750.0,4078
10,0.25,-4250.0,6130


In [10]:
using Gadfly

In [11]:
# plot(res, x=xCol, y=yCol, color=:value, Geom.rectbin)

- 

In [12]:
# Gadfly.plot(fullData, x="Time_to_Maturity", y="mnyness", Geom.hexbin(xbincount=100, ybincount=100))

In [13]:
# Gadfly.plot(fullData, x="Time_to_Maturity", y="mnyness", Stat.BinMeanStatistic, Geom.hexbin(xbincount=20, ybincount=20))

In [14]:
# Gadfly.plot(fullData, x="Time_to_Maturity", y="mnyness", Geom.histogram2d)

In [15]:
# Gadfly.plot(fullData, x="Time_to_Maturity", y="mnyness", Stat.histogram2d(xbincount = 50, ybincount = 50), Geom.rectbin)